# Part 1, Topic 1: Introduction to Clock Glitching (MAIN)

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *Microcontrollers and FPGAs have a number of operating conditions that must be met in order for the device to work properly. Outside of these conditions, devices will begin to malfunction, with more extreme violations causing the device to stop entirely or even become damaged. By going outside these operating conditions for very small amounts of time, we can cause a varitey of temporary malfunctions*

*In this lab, we'll explore clock glitching, which inserts short glitches into a device's clock. This will be used to get a target that's summing numbers in a loop to arrive at the wrong result.*

**LEARNING OUTCOMES:**

* Understand effects of clock glitching
* Exploring ChipWhisperer's glitch module
* Using clock glitching to disrupt a target's algorithm

## Clock Glitching Theory

Digital hardware devices almost always expect some form of reliable clock. We can manipulate the clock being presented to the device to cause unintended behaviour. We'll be concentrating on microcontrollers here, however other digital devices (e.g. hardware encryption accelerators) can also have faults injected using this technique.

Consider a microcontroller first. The following figure is an excerpt from the Atmel AVR ATMega328P datasheet:

![A2_1](img/Mcu-unglitched.png)

Rather than loading each instruction from FLASH and performing the entire execution, the system has a pipeline to speed up the execution process. This means that an instruction is being decoded while the next one is being retrieved, as the following diagram shows:

![A2_2](img/Clock-normal.png)

But if we modify the clock, we could have a situation where the system doesn't have enough time to actually perform an instruction. Consider the following, where Execute #1 is effectively skipped. Before the system has time to actually execute it another clock edge comes, causing the microcontroller to start execution of the next instruction:

![A2_3](img/Clock-glitched.png)

This causes the microcontroller to skip an instruction. Such attacks can be immensely powerful in practice. Consider for example the following code from `linux-util-2.24`:

```C
/*
 *   auth.c -- PAM authorization code, common between chsh and chfn
 *   (c) 2012 by Cody Maloney <cmaloney@theoreticalchaos.com>
 *
 *   this program is free software.  you can redistribute it and
 *   modify it under the terms of the gnu general public license.
 *   there is no warranty.
 *
 */

#include "auth.h"
#include "pamfail.h"

int auth_pam(const char *service_name, uid_t uid, const char *username)
{
    if (uid != 0) {
        pam_handle_t *pamh = NULL;
        struct pam_conv conv = { misc_conv, NULL };
        int retcode;

        retcode = pam_start(service_name, username, &conv, &pamh);
        if (pam_fail_check(pamh, retcode))
            return FALSE;

        retcode = pam_authenticate(pamh, 0);
        if (pam_fail_check(pamh, retcode))
            return FALSE;

        retcode = pam_acct_mgmt(pamh, 0);
        if (retcode == PAM_NEW_AUTHTOK_REQD)
            retcode =
                pam_chauthtok(pamh, PAM_CHANGE_EXPIRED_AUTHTOK);
        if (pam_fail_check(pamh, retcode))
            return FALSE;

        retcode = pam_setcred(pamh, 0);
        if (pam_fail_check(pamh, retcode))
            return FALSE;

        pam_end(pamh, 0);
        /* no need to establish a session; this isn't a
         * session-oriented activity...  */
    }
    return TRUE;
}
```

This is the login code for the Linux OS. Note that if we could skip the check of `if (uid != 0)` and simply branch to the end, we could avoid having to enter a password. This is the power of glitch attacks - not that we are breaking encryption, but simply bypassing the entire authentication module! 

### Glitch Hardware

The ChipWhisperer Glitch system uses the same synchronous methodology as its Side Channel Analysis (SCA) capture. A system clock (which can come from either the ChipWhisperer or the Device Under Test (DUT)) is used to generate the glitches. These glitches are then inserted back into the clock, although it's possible to use the glitches alone for other purposes (i.e. for voltage glitching, EM glitching).

The generation of glitches is done with two variable phase shift modules, configured as follows:

![A2_4](img/Glitchgen-phaseshift.png)

In CW-Husky there is one important difference: the phase shift 1 output is not inverted before it is ANDed with the phase shift 2 output.

The enable line is used to determine when glitches are inserted. Glitches can be inserted continuously (useful for development) or triggered by some event. The following figure shows how the glitch can be muxd to output to the Device Under Test (DUT).

![A2_5](img/Glitchgen-mux.png)

### Hardware Support: CW-Lite/Pro

The phase shift blocks use the Digital Clock Manager (DCM) blocks within the FPGA. These blocks have limited support for run-time configuration of parameters such as phase delay and frequency generation, and for maximum performance the configuration must be fixed at design time. The Xilinx-provided run-time adjustment can shift the phase only by about +/- 5nS in 30pS increments (exact values vary with operating conditions).

For most operating conditions this is insufficient - if attacking a target at 7.37MHz the clock cycle would have a period of 136nS. In order to provide a larger adjustment range, an advanced FPGA feature called Partial Reconfiguration (PR) is used. The PR system requires special partial bitstreams which contain modifications to the FPGA bitstream. These are stored as two files inside a "firmware" zip which contains both the FPGA bitstream along with a file called `glitchwidth.p` and a file called `glitchoffset.p`. If a lone bitstream is being loaded into the FPGA (i.e. not from the zip-file), the partial reconfiguration system is disabled, as loading incorrect partial reconfiguration files could damage the FPGA. This damage is mostly theoretical, more likely the FPGA will fail to function correctly.

If in the course of following this tutorial you find the FPGA appears to stop responding (i.e. certain features no longer work correctly), it could be the partial reconfiguration data is incorrect.

We'll look at how to interface with these features later in the tutorial.

### Hardware Support: CW-Husky

The clock-generation logic in Husky's 7-series FPGA is considerably different than the 6-series FPGAs used in CW-Lite/Pro. The DCM is gone and replaced by the much more powerful (and power hungry...) Mixed Mode Clock Manager (MMCM). In particular for our glitching application, MMCMs allow fine phase shift adjustments over an unlimited range, in steps as small as 15ps. And all this without having to dynamically reconfigure the FPGA bitfile! For this reason, the format for specifying the glitch offset and width is different from what it was for CW-Lite/Pro. Instead of specifiying a percentage of the source clock period, you now specify the actual number of phase shift steps. The duration of one phase shift step is 1/56 of the MMCM VCO clock period, which can itself be configured to be anyhwere in the range from 600 MHz to 1200 MHz (via `scope.clock.pll.update_fpga_vco()`).

While the MMCM is more powerful than the DCM with respect to its features, it also requires a lot more power. For this reason, the glitch generation circuitry is disabled by default and must be explicitly turned on. Fear not, Husky also uses Xilinx's XADC module to continuously monitor its temperature, and all MMCMs are automatically turned off at when the temperature starts getting too high, well below dangerous levels are reached (run `scope.XADC` to see all its statistics and settings).


In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_SAM4S'
SS_VER = 'SS_VER_2_1'

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

In [ ]:
fw_path = "../../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
if SS_VER=='SS_VER_2_1':
    target.reset_comms()

We'll probably crash the target a few times while we're trying some glitching. Create a function to reset the target:

In [ ]:
if PLATFORM == "CWLITEXMEGA":
    def reboot_flush():            
        scope.io.pdic = False
        time.sleep(0.1)
        scope.io.pdic = "high_z"
        time.sleep(0.1)
        #Flush garbage too
        target.flush()
else:
    def reboot_flush():            
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = "high_z"
        time.sleep(0.05)
        #Flush garbage too
        target.flush()

## Communication

For this lab, we'll be introducing a new method: `target.simpleserial_read_witherrors()`. We're expecting a simpleserial response back; however, glitch will often cause the target to crash and return an invalid string. This method will handle all that for us. It'll also tell us whether the response was valid and what the error return code was. Use as follows:

In [ ]:
#Do glitch loop
target.simpleserial_write('g', bytearray([]))

val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=10)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

#print(bytearray(val['full_response'].encode('latin-1')))
print(val)

## Target Firmware

For this lab, our goal is to get the following code to preduce an incorrect result:

```C
uint8_t glitch_loop(uint8_t* in)
{
    volatile uint16_t i, j;
    volatile uint32_t cnt;
    cnt = 0;
    trigger_high();
    for(i=0; i<50; i++){
        for(j=0; j<50; j++){
            cnt++;
        }
    }
    trigger_low();
    simpleserial_put('r', 4, (uint8_t*)&cnt);
    return (cnt != 2500);
}
```

As you can see, we've got a simple loop. This is a really good place to start glitching for 2 reasons:

1. We've got a really long portion of time with a lot of instructions to glitch. In contrast, with the Linux example we're be trying to target a single instruction.

1. For some glitching scenarios, we're looking for a pretty specific glitch effect. In the Linux example, we might be banking on the glitch causing the target to skip an instruction instead of corrupting the comparison since that's a lot more likely to get us where we want in the code path. For this simple loop calculation, pretty much any malfunction will show up in the result.

## Glitch Module

All the settings/methods for the glitch module can be accessed under `scope.glitch`. As usual, documentation for the settings and methods can be accessed on [ReadtheDocs](https://chipwhisperer.readthedocs.io/en/latest/api.html) or with the python `help` command:

In [ ]:
help(scope.glitch)

#### We'll first go over settings that differ between the CW Husky and the CW Lite/Pro:
* clk_src

> The clock signal that the glitch DCM is using as input. Can be set to "target" or "clkgen" In this case, we'll be providing the clock to the target, so we'll want this set to "clkgen".

> On CW Husky, a separate PLL is used to clock the glitch module instead of the clkgen module. The equivalent setting here for "clkgen" is "pll"
* offset

> Where in the output clock to place the glitch. Can be in the range `[-48.8, 48.8]`. Often, we'll want to try many offsets when trying to glitch a target.

> On CW Husky, the range will depend on frequency of the PLL used to drive the glitch module (settable which can be configured to be anyhwere in the range from 600 MHz to 1200 MHz via `scope.clock.pll.update_fpga_vco()`), but, when the glitch module is active, the range will be `[0, scope.glitch.phase_shift_steps]`.
* width

> How wide to make the glitch. Can be in the range `[-50, 50]`, though there is no reason to use widths < 0. Wider glitches more easily cause glitches, but are also more likely to crash the target, meaning we'll often want to try a range of widths when attacking a target.

> Like offset, the range will be `[0, scope.glitch.phase_shift_steps]`.

#### These settings, on the other hand, are the same between the Husky and the Lite/Pro:

* output

> The output produced by the glitch module. For clock glitching, clock_xor is often the most useful option, as this inverts the clock during the glitch.
* ext_offset

> The number of clock cycles after the trigger to put the glitch.
* repeat

> The number of clock cycles to repeat the glitch for. Higher values increase the number of instructions that can be glitched, but often increase the risk of crashing the target.

* trigger_src

> How to trigger the glitch. For this tutorial, we want to automatically trigger the glitch from the trigger pin only after arming the ChipWhipserer, so we'll use `ext_single`

In addition, we'll need to tell ChipWhipserer to use the glitch module's output as a clock source for the target by setting `scope.io.hs2 = "glitch"`. We'll also setup a large `repeat` to make glitching easier.

## CW Glitch Controller

To make creating a glitch loop easier, ChipWhisperer includes a glitch controller. We'll start of by initializing it with different potential results of the attack. You define these to be whatever you want, but often three groups are sufficient:

1. `"success"`, where our glitch had the desired effect
1. `"reset"`, where our glitch had an undesirable effect. Often, this effect is crashing or resetting the target, which is why we're calling it `"reset"`
1. `"normal"`, where you glitch didn't have a noticable effect.

We also need to tell it what glitch parameters we want to scan through, in this case width and offset. We'll also add a "tries" parameter which, as the name suggests, is just there to try each setting multiple times:

In [ ]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset", "tries"])

One of the niceties of the glitch controller is that it can display our current settings. This will update in real time as we use the glitch controller!

In [ ]:
gc.display_stats()

We can also make a settings map that can also update in realtime as well:

In [ ]:
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None})

Here `plotdots` is a dictionary that specifies how you want to plot each group. In this case, we're plotting `"success"` as a green `+` (`"+g"`), `"reset"` as a red `x` (`"xr"`), and we won't be plotting glitch attempts where nothing abnormal happens (`None`)

This plot will auto update its bounds as points are added. If you want to specify the axis bounds, you can do so as follows:

```python
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, x_bound=(-48, 48), y_bound=(-48, 48))
```

You can also select which parameters you want to use for x and y, either by index, or by its name:

```python
# will flip width and offset axes
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, x_index=1, y_index=0)
# or
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, x_index="offset", y_index="width")

```

You can set ranges for each glitch setting:

In [ ]:
gc.set_range("width", -5, 5)
gc.set_range("offset", -5, 5)

Each setting moves from min to max based on the global step:

In [ ]:
gc.set_global_step([5.0, 2.5])

We can print out all the glitch settings to see how this looks:

In [ ]:
for glitch_setting in gc.glitch_values():
    print("offset: {:4.1f}; width: {:4.1f}".format(glitch_setting[1], glitch_setting[0]))

You can tell the glitch controller when you've reached a particular result state like so:

In [ ]:
#gc.add("reset", (scope.glitch.width, scope.glitch.offset)) or simply gc.add("reset")
#gc.add("success", (scope.glitch.width, scope.glitch.offset)) or simply gc.add("success")

As of ChipWhisperer 5.7, you can skip the glitch width and glitch offset parameters. In this case, the glitch controller will use its internal values for the coordinates. Note that due to rounding, this will usually be a bit different from the actual hardware value on the Lite/Pro; however, the values will still correspond to the correct settings on your ChipWhisperer.

We'll start off with the following settings. It's usually best to use "clock_xor" with clock glitching, which will insert a glitch if the clock is high or the clock is low.

For CW-Husky, we must first explicitly turn on the glitch circuitry (it is off by default for power savings):

In [ ]:
if scope._is_husky:
    scope.glitch.enabled = True

We'll start off with the following settings. It's usually best to use "clock_xor" with clock glitching, which will insert a glitch if the clock is high or the clock is low.

In [ ]:
#Basic setup
# set glitch clock
if scope._is_husky:
    scope.glitch.clk_src = "pll"
else:
    scope.glitch.clk_src = "clkgen" 

scope.glitch.output = "clock_xor" # glitch_out = clk ^ glitch
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called

scope.io.hs2 = "glitch"  # output glitch_out on the clock line
print(scope.glitch)

These settings are often a good starting point for all clock glitching, so, new with ChipWhisperer 5.7, we've got a method that sets all of this up for you:

In [ ]:
scope.cglitch_setup()

You should have all you need to construct your glitch loop. We'll get you started, but the rest is up to you! Also, some stuff to keep in mind:

* You'll need to detect crashes, successful glitches, and normal returns from the target. Don't be afraid to experiment with the loop: you can always restart it and rerun the code.
* You can cover a larger set of glitch settings by starting with large glitch controller steps to get idea where some interesting locations are, then repeating the glitch loop with small steps in interesting areas. Where there's one successful glitch, there's probably more!
* You can speed up your glitch campaign substantially by only plotting crashes and successes, since they're typically much rarer than normal behaviour in the target
* On CW-Husky, glitch offset and width are specified in number of phase shift steps, whereas on CW-Lite/Pro, they are specified in percentage of clock period. The code provided below sets appropriate starting ranges for each case. Run `help(scope.glitch)` to understand this better.

In [ ]:
from tqdm.notebook import trange
import struct

# width and offset numbers have a very different meaning for Husky vs Lite/Pro;
# see help(scope.glitch) for details
num_tries = 1
gc.set_range("tries", 1, num_tries)
if scope._is_husky:
    gc.set_range("width", 0, scope.glitch.phase_shift_steps//2)
    gc.set_range("offset", 0, scope.glitch.phase_shift_steps)
    gc.set_global_step([100]) # reduce if you don't get any glitches
    scope.adc.lo_gain_errors_disabled = True
    scope.adc.clip_errors_disabled = True
else:
    gc.set_range("width", 0, 48)
    gc.set_range("offset", -48, 48)
    gc.set_global_step([8, 4, 2, 1])
    
scope.glitch.repeat = 1
gc.set_step("tries", 1)
scope.adc.timeout = 0.1
gc.set_range("ext_offset", 0, 40)
gc.set_step("ext_offset", 1)

reboot_flush()
broken = False
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    scope.glitch.ext_offset = glitch_setting[2]
    # ###################
    # Add your code here
    # ###################
    #raise NotImplementedError("Add your code here, and delete this.")

    # ###################
    # START SOLUTION
    # ###################
    if scope.adc.state:
        # can detect crash here (fast) before timing out (slow)
        print("Trigger still high!")
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()

    scope.arm()

    #Do glitch loop
    target.simpleserial_write('g', bytearray([]))

    ret = scope.capture()
    
    loff = scope.glitch.offset
    lwid = scope.glitch.width

    if ret:
        print('Timeout - no trigger')
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()

    else:
        val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=10, timeout=50)#For loop check
        if val['valid'] is False:
            gc.add("reset")
        else:

            #print(val['payload'])
            if val['payload'] is None:
                print(val['payload'])
                continue #what
            #gcnt = struct.unpack("<b", val['payload'])[0] #for code-flow check
            gcnt = struct.unpack("<I", val['payload'])[0]

            #print(gcnt)                
            # for table display purposes
            #if gnt != 0: #for code-flow check
            if gcnt != 2500: #for loop check
                broken = True
                gc.add("success")
                print(val['payload'])
                print("🐙", end="")
                break # <-- remove this to try for multiple glitches
            else:
                gc.add("normal")
    # ###################
    # END SOLUTION
    # ###################

print("Done glitching")

### Results

In addition to plotting, the glitch controller also has the capability to return results as a list that groups paramters and results. These results give both the number of each result, as well as the rate of each result:

In [ ]:
gc.calc()

You can also get results back with some parameters ignored. Results from parameters that now match will be grouped. This is particularly useful with something like the `"tries"` parameter, as you don't typically care whether a glitch was successful on your first, second, or third attempt:

In [ ]:
results = gc.calc(ignore_params="tries")
results

Finally, `calc()` can also sort by different results. A common use for this is to sort by success rate:

In [ ]:
results = gc.calc(ignore_params="tries", sort="success_rate")
results

### Plotting Glitch Results

We can replot our glitch map using the `plot_2d()` method. Settings are similar to `glitch_plot()`. If `plotdots` are not specified, the same ones as the `glitch_plot()` will be used.

`plot_2d()` also has the advantage of displaying an alpha channel. The hover tool, which allows you to see the coordinates by mousing over a plot point, has a separate button for each group on the right toolbar. This allows you to see, for example, only the hover information for successes by turning the reset hover off. The hover tool will also show the rate of occurrence for the group.

In [ ]:
gc.plot_2d()

Make sure you write down those glitch settings, since we'll be using for the rest of the glitching labs! In fact, we'll be using a lot of the general code structure here for the rest of the labs, with the only big changes being:

### Repeat

This lab used a pretty large repeat value. Like the name suggests, this setting controls how many times the glitch is repeated (i.e. a repeat value of 5 will place glitches in 5 consecutive clock cycles). Consider that each glitch inserted has a chance to both cause a glitch or crash the device. This was pretty advantageous for this lab since we had a lot of different spots we wanted to place a glitch - using a high repeat value increased our chance for a crash, but also increased our chance for a successful glitch. For an attack where we're targeting a single instruction, we don't really increase our glitch chance at all, but still have the increased crash risk. Worse yet, a successful glitch in a wrong spot may also cause a crash! It is for that reason that it's often better to use a low repeat value when targeting a single instruction.

### Ext Offset

The ext offset setting controls a delay between the trigger firing and the glitch being inserted. Like repeat, it's base on whole clock cycles, meaning an ext offset of 10 will insert a glitch 10 cycles after the trigger fires. We didn't have to worry about this setting for this lab since the large repeat value was able to take us into the area we wanted. This won't be true for many applications, where you'll have to try glitches at a large variety of ext_offsets.

### Success, Reset, and Normal

These three result states are usually enough to describe most glitch results. What constitues a success, however, will change based on what firmware you're attacking. For example, if we were attacking the Linux authentication, we might base success on a check to see whether or not we're root.

In [ ]:
#scope.dis()
#target.dis()

In [ ]:
assert broken is True